##### This notebook contains a quickstart guide to using OpenXAI, covering:
- `(0) Preamble and data/model selection`
- `(1) Loading preprocessed datasets`
- `(2) Loading pretrained models`
- `(3) Configuring and using explanation methods`
- `(4) Configuring and using evaluation metrics on the explanations generated`

##### Full model training, explanation, and evaluation pipelines can be found at:
- `train_models.py`
- `generate_explanations.py`
- `evaluate_metrics.py`

##### Each of which parses parameters from:
- `experiment_config.json`

*This notebook is a lightweight alternative to the full pipelines, intended for users who want to get started quickly or customize their own pipeline.*

### (0) Preamble

In [83]:
# Utils
import torch
import numpy as np
from openxai.experiment_utils import print_summary, load_config, fill_param_dict
from openxai.explainers.perturbation_methods import get_perturb_method

# ML models
from openxai.model import LoadModel

# Data loaders
from openxai.dataloader import return_loaders, return_train_test_inputs

# Explanation models
from openxai.explainer import Explainer

# Evaluation methods
from openxai.evaluator import Evaluator

In [2]:
# Choose the model and the data set you wish to generate explanations for
n_test_samples = 10
data_name = 'adult' # must be one of ['adult', 'compas', 'gaussian', 'german', 'gmsc', 'heart', 'heloc', 'pima']
model_name = 'lr'    # must be one of ['lr', 'ann']

### (1) Data Loaders

In [81]:
# Get training and test loaders
trainloader, testloader = return_loaders(data_name=data_name,
                                           download=True,
                                           batch_size=n_test_samples)
inputs, labels = next(iter(testloader))
labels = labels.type(torch.int64)

# Get full train/test FloatTensors and feature metadata
X_train, X_test, feature_metadata = return_train_test_inputs(data_name, float_tensor=True, return_feature_metadata=True)

### (2) Load a pretrained ML model

In [57]:
# Load pretrained ml model
model = LoadModel(data_name=data_name,
                  ml_model=model_name,
                  pretrained=True)
print_summary(model, trainloader, testloader)
preds = model(inputs.float()).argmax(1)
print(f'First 10 predictions: {preds[:10]}')

Proportion of Class 1:
	Test Preds:	0.2075
	Test Set:	0.2479
Test Accuracy: 0.8325
Train Accuracy: 0.8349
First 10 predictions: tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])


### (3) Choose an explanation method

#### I. Explanation method with config hyperparameters (LIME)

In [58]:
# Choose explainer
method = 'lime'

# Load config parameters for the explainer
param_dict = load_config('experiment_config.json')['explainers'][method]

# # If LIME/IG, then provide X_train
param_dict = fill_param_dict(method, param_dict, X_train)
print(f'{method.upper()} Parameters\n\n' +'\n'.join([f'{k}: {v}' for k, v in param_dict.items()]))

LIME Parameters

n_samples: 1000
kernel_width: 0.75
std: 0.1
mode: tabular
sample_around_instance: True
discretize_continuous: False
seed: 0
data: tensor([[0.4384, 0.1733, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.1233, 0.1169, 0.5333,  ..., 1.0000, 1.0000, 1.0000],
        [0.2877, 0.0135, 0.5333,  ..., 0.0000, 1.0000, 1.0000],
        ...,
        [0.1096, 0.2245, 0.3333,  ..., 1.0000, 1.0000, 1.0000],
        [0.4658, 0.0893, 0.8000,  ..., 0.0000, 1.0000, 1.0000],
        [0.2877, 0.0263, 0.8000,  ..., 1.0000, 1.0000, 1.0000]])


In [59]:
# Compute explanations
lime = Explainer(method, model, param_dict)
lime_exps = lime.get_explanation(inputs, preds).detach().numpy()
print(lime_exps[0])

[-0.0011376  -0.00234992 -0.01240377 -0.03825278 -0.01005829  0.00205742
  0.00217722  0.00086493  0.00163225 -0.00117567  0.00221764  0.00232039
 -0.00024308]


#### II: Explanation method with default hyperparameters (LIME)

In [60]:
# Choose explainer
method = 'lime'

# Pass empty dict to use default parameters
param_dict = {}

# # If LIME/IG, then provide X_train
param_dict = fill_param_dict(method, {}, X_train)
print(f'{method.upper()} Parameters\n\n' +'\n'.join([f'{k}: {v}' for k, v in param_dict.items()]))
print('Remaining parameters are set to their default values')

LIME Parameters

data: tensor([[0.4384, 0.1733, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [0.1233, 0.1169, 0.5333,  ..., 1.0000, 1.0000, 1.0000],
        [0.2877, 0.0135, 0.5333,  ..., 0.0000, 1.0000, 1.0000],
        ...,
        [0.1096, 0.2245, 0.3333,  ..., 1.0000, 1.0000, 1.0000],
        [0.4658, 0.0893, 0.8000,  ..., 0.0000, 1.0000, 1.0000],
        [0.2877, 0.0263, 0.8000,  ..., 1.0000, 1.0000, 1.0000]])
Remaining parameters are set to their default values


In [66]:
# Compute explanations
lime = Explainer(method, model, param_dict)
lime_exps = lime.get_explanation(inputs.float(), preds).detach().numpy()
print(lime_exps[0])

[-0.01484873 -0.00360259 -0.05157194 -0.15891102 -0.03377749 -0.01719308
  0.01170494  0.00064746  0.03330764  0.00893782  0.01174643  0.00024131
 -0.00099279]


#### III: Explanation method with default hyperparameters (IG)

In [67]:
# Choose explainer
method = 'ig'

# If LIME/IG, then provide X_train
param_dict = fill_param_dict('ig', {}, X_train)

# Compute explanations
ig = Explainer('ig', model, param_dict)
ig_exps = ig.get_explanation(inputs.float(), preds).detach().numpy()
print(ig_exps[0])

[-0.01484873 -0.00360259 -0.05157194 -0.15891102 -0.03377749 -0.01719308
  0.01170494  0.00064746  0.03330764  0.00893782  0.01174643  0.00024131
 -0.00099279]


#### IV: Explanation method with additional hyperparameters (SHAP)

In [69]:
# Choose explainer
method = 'shap'

# Override default parameters for certain hyperparameters
param_dict = {'n_samples': 1000, 'seed': 0}

# Compute explanations
shap = Explainer(method, model, param_dict)
shap_exps = shap.get_explanation(inputs.float(), preds).detach().numpy()
print(shap_exps[0])

[-0.00378722 -0.01268718 -0.12357364  0.00807462  0.00349009 -0.01655754
  0.03306515 -0.0088843   0.05668441  0.02878716  0.04120433  0.0033168
  0.00390152]


### (4) Choose an evaluation metric

In [87]:
from openxai.evaluator import ground_truth_metrics, prediction_metrics, stability_metrics
print('Ground truth metrics: ', ground_truth_metrics)
print('Prediction metrics: ', prediction_metrics)
print('Stability metrics: ', stability_metrics)

Ground truth metrics:  ['PRA', 'RC', 'FA', 'RA', 'SA', 'SRA']
Prediction metrics:  ['PGU', 'PGI']
Stability metrics:  ['RIS', 'RRS', 'ROS']


#### I. Ground Truth Faithfulness Metrics (PRA, RC, FA, RA, SA, SRA)

In [97]:
# Choose one of ['PRA', 'RC', 'FA', 'RA', 'SA', 'SRA']
metric = 'FA'  

# Load config
param_dict = load_config('experiment_config.json')['evaluators']['ground_truth_metrics']
param_dict['explanations'] = lime_exps
if metric in ['FA', 'RA', 'SA', 'SRA']:
    param_dict['predictions'] = preds  # flips ground truth according to prediction
elif metric in ['PRA', 'RC']:
    del param_dict['k'], param_dict['AUC']  # not needed for PRA/RC

# Print final parameters
params_preview = [f'{k}: array of size {v.shape}' if hasattr(v, 'shape') else f'{k}: {v}' for k, v in param_dict.items()]
print(f'{metric.upper()} Parameters\n\n' +'\n'.join(params_preview))

FA Parameters

k: 0.25
AUC: True
explanations: array of size (10, 13)
predictions: array of size torch.Size([10])


#### II. Predictive Faithfulness Metrics (PGU, PGI)

In [98]:
# Choose one of ['PGU', 'PGI']
metric = 'PGU'

# Load config
param_dict = load_config('experiment_config.json')['evaluators']['prediction_metrics']
param_dict['inputs'] = X_test
param_dict['explanations'] = lime_exps
param_dict['feature_metadata'] = feature_metadata
param_dict['perturb_method'] = get_perturb_method(param_dict['std'], data_name)
del param_dict['std']

# Print final parameters
params_preview = [f'{k}: array of size {v.shape}' if hasattr(v, 'shape') else f'{k}: {v}' for k, v in param_dict.items()]
print(f'{metric.upper()} Parameters\n\n' +'\n'.join(params_preview))

PGU Parameters

k: 0.25
AUC: True
n_samples: 100
seed: -1
n_jobs: -1
inputs: array of size torch.Size([9045, 13])
explanations: array of size (10, 13)
feature_metadata: ['c', 'c', 'c', 'c', 'c', 'c', 'd', 'd', 'd', 'd', 'd', 'd', 'd']
perturb_method: <openxai.explainers.perturbation_methods.NormalPerturbation object at 0x291491d90>


#### III. Stability Metrics (RIS, RRS, ROS)

In [104]:
# Choose one of ['RIS', 'RRS', 'ROS']
metric = 'RIS'

# Initialize explainer for stability metrics
method = 'grad'
exp_param_dict = load_config('experiment_config.json')['explainers'][method]
exp_param_dict = fill_param_dict(method, exp_param_dict, X_train)  # if LIME/IG
explainer = Explainer(method, model, exp_param_dict)


# Load config
param_dict = load_config('experiment_config.json')['evaluators']['stability_metrics']
param_dict['inputs'] = X_test
param_dict['explainer'] = explainer
param_dict['perturb_method'] = get_perturb_method(param_dict['std'], data_name)
param_dict['feature_metadata'] = feature_metadata
del param_dict['std']

# Print final parameters
params_preview = [f'{k}: array of size {v.shape}' if hasattr(v, 'shape') else f'{k}: {v}' for k, v in param_dict.items()]
print(f'{metric.upper()} Parameters\n\n' +'\n'.join(params_preview))

RIS Parameters

n_samples: 1000
n_perturbations: 100
p_norm: 2
seed: -1
n_jobs: -1
inputs: array of size torch.Size([9045, 13])
explainer: <openxai.explainers.catalog.grad.grad.Gradient object at 0x17391ed30>
perturb_method: <openxai.explainers.perturbation_methods.NormalPerturbation object at 0x292923280>
feature_metadata: ['c', 'c', 'c', 'c', 'c', 'c', 'd', 'd', 'd', 'd', 'd', 'd', 'd']


### (5) Evaluate the explanations

In [115]:
# Evaluate the metric across the test inputs/explanations
evaluator = Evaluator(model, metric)
score, mean_score = evaluator.evaluate(**param_dict)

In [117]:
# Print results
std_err = np.std(score) / np.sqrt(len(score))
print(f"{metric}: {mean_score:.2f}\u00B1{std_err:.2f}")
if metric in stability_metrics:
    log_mu, log_std = np.log(mean_score), np.log(std_err)
    print(f"log({metric}): {log_mu:.2f}\u00B1{log_std:.2f}")

RIS: 42.01±0.32
log(RIS): 3.74±-1.13
